In [0]:
!source scgen/bin/activate
!pip install scvi-tools
!pip install git+https://github.com/theislab/scgen.git

In [0]:
import sys
#if branch is stable, will install via pypi, else will install from source
branch = "stable"
IN_COLAB = "google.colab" in sys.modules

if IN_COLAB and branch == "stable":
    !pip install --quiet scgen[tutorials]
elif IN_COLAB and branch != "stable":
    !pip install --quiet --upgrade jsonschema
    !pip install --quiet git+https://github.com/theislab/scgen@$branch#egg=scgen[tutorials]

### Loading Train Data

In [0]:
import logging
import scanpy as sc
import sklearn
import scgen
import numpy as np

In [0]:
np.random.seed(42)

In [0]:
adata = sc.read("./tests/data/train_kang.h5ad",
                backup_url='https://drive.google.com/uc?id=1r87vhoLLq6PXAYdmyyd89zG90eJOFYLk')

In [0]:
#create test train split
train_size = int(adata.shape[0]*0.8)
test_size = adata.shape[0] - train_size


In [0]:
train_idx = np.random.choice(adata.obs_names, size = train_size, replace = False)

In [0]:
train_set = adata[adata.obs_names.isin(train_idx.tolist())].copy()
test_set = adata[~adata.obs_names.isin(train_idx.tolist())].copy()

In [0]:
train_set = adata[adata.obs_names.isin(train_idx.tolist())].copy()
test_set = adata[~adata.obs_names.isin(train_idx.tolist())].copy()

In [0]:
scgen.SCGEN.setup_anndata(train_set, batch_key="condition", labels_key="cell_type")

In [0]:
model = scgen.SCGEN(train_set)
model.save("../saved_models/model_perturbation_prediction.pt", overwrite=True)

In [0]:
model.train(
    max_epochs=100,
    batch_size=32,
    early_stopping=True,
    early_stopping_patience=25
)

## Prediction

In [0]:
pred, delta = model.predict(
    ctrl_key='train',
    stim_key='test',
    celltype_to_predict='CD4T'
)
pred.obs['condition'] = 'pred'

In [0]:
latent_adata.X

In [0]:
adata.X.A